In [3]:
from influxdb import InfluxDBClient
import uuid
import random
import time

In [4]:
client = InfluxDBClient(host='192.52.35.50', port=8086)

In [5]:
client.create_database('smartHomeTest')

In [6]:
random.seed()


measurement_name = 'sh1'

persons = [ 
    "Leo",
    "Oli"
]

locations = [
    "Wohnzimmer",
    "Küche",
    "Bad",
    "Schlafzimmer"
]

#[name,standardverbrauch idle, spitzenverbrauch,DurchschnittlicheDauerSpitzenverbrauchInTicks]
verbraucher = [
    ["Server",100,300,2],
    ["Kühlschrank",50,100,3],
    ["Mixer",0,3000,1],
    ["Licht",20,100,36],
    ["Lüftung",20,40,12],
    ["Fernseher",1,100,15],
    ["PC",5,300,20],
    ["Ofen",0,3000,15]
]

#temperatursensor
#Humiditysensor
#heizungssteuerung

heizung_stby = 17
heizung_normal = 22

In [ ]:
#creates a dataFrame, that can be supplied to InfluxDB
#tagSet needs to be supplied in ["tagName1=tagValue1","tagName2=tagValue2"] Optional
#fieldSet needs to be supplied in ["fieldName1=fieldValue1","fieldName2=fieldValue2"]
def createInsertDataPoint(tagSet, fieldSet,measurementName = measurement_name, timestamp = time.time() * 1000):
    returnDF = "" + measurementName 
    
    for tagElement in tagSet:
        returnDF = returnDF + "," + tagElement
    
    returnDF = returnDF + " "
    
    first=True;
    
    for fieldElement in fieldSet:
        if first:
            returnDF = returnDF + fieldElement
            first = False
        else:
            returnDF = returnDF + "," + fieldElement
    
    returnDF = returnDF + " "
    
    returnDF = returnDF + str(timestamp)

    return returnDF

In [ ]:
#the sum of x to 1 is close to 0.5. That means, there will be a 50% chance
#until it is the first time ticked to 1
#every next tip has a probability of 0.25
class halfProbability:
    
    def __init__(self, half):
        self.half = half
    def tick (self):
        if self.half > 1:
            self.half = self.half - 1
        return 1.0/2.0 * 1.0/2.0**float(self.half)

In [ ]:
#this is the person cell, that keeps track of the plausability of the persons that are at home

#there will be objects stored in it [string name, bool yes/no, int sinceTicks, probabilisticObject]
personsCurrentlyAtHomeSince = []

#parameter:

# persons will be averagely 4 h at home, or away. That will be 48 ticks
homeHalfTime = 48

for person in persons:
    personsCurrentlyAtHomeSince.append([person,True,0,halfProbability(homeHalfTime)])

def manager_persons(introduceAnnormaly = False):
    
    for person in personsCurrentlyAtHomeSince:
        person[2] += 1 #adds one to the ticks, that they are at home
        
        #print(person)
        
        # now the probabilistic stuff:
        #grab our probabilistic object
        rand = person[3].tick()
        #print(str(rand))
        if random.random() < rand:            
            print(createInsertDataPoint(
                tagSet=["Person="+person[0]],
                fieldSet=["isHome=" + str(person[1])]))
            
            person[1] = not person[1]
            person[2] = 0
            person[3] = halfProbability(homeHalfTime)
        

In [ ]:
#anormalies
def openWindow(temp):
    temp -= heizung_stby * 0.05 #drop temp by 5% of stby temp
    return temp

def heatingToHot(temp):
    temp += heizung_normal * 0.03 #heat 3%
    return temp

def none(temp):
    random.random()#do nothing
    return temp

#in there are [temp,currentannormaly,howLongShouldTheAnormalyLast,location]
currentTemps = []
for room in locations:
    currentTemps.append([21.0,none,0,room])
    
heatingAnormalyTimer = 20
heatingAnormalies = [openWindow,heatingToHot,none]


def manage_heating(introduceAnnormaly = False):
    atLeastOneIsHome = False
    for person in personsCurrentlyAtHomeSince:
        if person[1]:
            atLeastOneIsHome = True
    for currentTemp in currentTemps:
        if atLeastOneIsHome:
            if currentTemp[0] < heizung_normal:
                currentTemp[0] += heizung_normal * 0.02 #heat 2% per tick
        else: #no one is home
            if currentTemp[0] > heizung_stby:
                currentTemp[0] -= heizung_stby * 0.02 #cool 2% per tick
        
        #now handle the anormalies:
        if currentTemp[2] > 0:
            currentTemp[2] -= 1
            currentTemp[0] = currentTemp[1](currentTemp[0])#start the anormaly function and safe temp
            
            
    if introduceAnnormaly:
        #chose a random room and start an anormaly
        roomA = random.choice(currentTemps)
        roomA[1] = random.choice(heatingAnormalies)
        roomA[2] = heatingAnormalyTimer
        print("Introducing an Anormaly in the heating system: " + str(roomA[3]))
        
    for temp in currentTemps:
        #print("current temp: " + str(temp))
        print(createInsertDataPoint(
                tagSet=["Room="+temp[3]],
                fieldSet=["Temperature=" + str(temp[0])]))
        

In [ ]:
#[verbraucherName,standardverbrauch,spitzenverbrauch,peakDauer],location,aktuellerVerbrauch,peakTicks
currentEnergy = []

for verbr in verbraucher:
    currentEnergy.append([verbr,random.choice(locations),0,0])
    
def manage_energy(introduceAnnormaly = False):
    personsAtHome = 0;
    for person in personsCurrentlyAtHomeSince:
        if person[1]:
            personsAtHome += 1
    
    ItemsInUse = 0 # we assume, that per person home, is one item in use
    for verbr in currentEnergy:
        if verbr[3] > 0:
            ItemsInUse += 1
    
    if personsAtHome > ItemsInUse:
        verb = random.choice(currentEnergy)
        verb[3] = verb[0][3] # set a new device to use the spitzenverbrauch
        verbr[2] = verbr[0][2] #set to spitzenverbrauch
        
    for verbr in currentEnergy:
        if verbr[3] > 0: #ticks left
            verbr[3] -= 1#one tick less
        else:
            verbr[2] = verbr[0][1] #set to idle verbrauch
    
    if introduceAnnormaly:
        verb = random.choice(currentEnergy)
        verb[3] = verb[0][3] * 2 #set to wait 2 times the normal spitzenverbrauch ticks, befor using normal again
        if(random.random() > 0.5):
            verb[3] = verb[0][3] # set a new device to use the spitzenverbrauch
        else:
            verb[3] = 0 # set to 0, as an anormaly
    
    for verbr in currentEnergy:
        print(createInsertDataPoint(
            tagSet=["Raum="+str(verbr[1]),"Verbraucher="+str(verbr[0][0])],
            fieldSet=["Verbrauch=" + str(verbr[2])]))
    
    

In [ ]:
#tick engine

# 1 Second real time is 5 minutes simulation time

tickSubscribers = [
    manager_persons,
    manage_heating,
    manage_energy
    
]

for i in range(0,100):
    time.sleep(0.5)
    for func in tickSubscribers:
        anormaly = False
        if (random.random() > 0.99):
            print("Anormaly introduced")
            anormaly = True
        func(anormaly)


In [ ]:

measurement_name = 'sh1'
number_of_points = 250000
data_end_time = int(time.time() * 1000) #milliseconds

location_tags = [ "reservoir",
"orchard",
"vineyard",
"quarry",
"hospital"]

fruit_tags = [ "apple",
"banana",
"cantaloupe",
"cherry",
"coconut",
"durian"]

id_tags = []
for i in range(100):
    id_tags.append(str(uuid.uuid4()))

data = []
data.append("{measurement},location={location},fruit={fruit},id={id} x={x},y={y},z={z}i {timestamp}"
            .format(measurement=measurement_name,
                    location=random.choice(location_tags),
                    fruit=random.choice(fruit_tags),
                    id=random.choice(id_tags),
                    x=round(random.random(),4),
                    y=round(random.random(),4),
                    z=random.randint(0,50),
                    timestamp=data_end_time))
current_point_time = data_end_time
for i in range(number_of_points-1):
    current_point_time = current_point_time - random.randint(1,100)
    data.append("{measurement},location={location},fruit={fruit},id={id} x={x},y={y},z={z}i {timestamp}"
                .format(measurement=measurement_name,
                        location=random.choice(location_tags),
                        fruit=random.choice(fruit_tags),
                        id=random.choice(id_tags),
                        x=round(random.random(),4),
                        y=round(random.random(),4),
                        z=random.randint(0,50),
                        timestamp=current_point_time))

client_write_start_time = time.perf_counter()

client.write_points(data, database='writetest', time_precision='ms', batch_size=10000, protocol='line')

client_write_end_time = time.perf_counter()

print("Client Library Write: {time}s".format(time=client_write_end_time - client_write_start_time))
